In [ ]:
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler
import pandas as pd
import numpy as np
import joblib
import json
import time

def criar_dataset_multi_step(series, look_back=10, passo=1, adicionar_features=True):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        # Extrair a sequência base
        sequencia = series[i:i + look_back].flatten()
        
        if adicionar_features:
            # Adicionar características para detecção de picos
            media_movel = np.mean(sequencia)
            std_movel = np.std(sequencia)
            max_local = np.max(sequencia)
            min_local = np.min(sequencia)
            amplitude = max_local - min_local
            tendencia = sequencia[-1] - sequencia[0]
            
            # Derivada (taxa de mudança)
            derivada = np.diff(sequencia)
            derivada_mean = np.mean(derivada) if len(derivada) > 0 else 0
            derivada_std = np.std(derivada) if len(derivada) > 0 else 0
            
            # Segunda derivada (aceleração)
            segunda_derivada = np.diff(derivada) if len(derivada) > 1 else np.array([0])
            segunda_derivada_mean = np.mean(segunda_derivada) if len(segunda_derivada) > 0 else 0
            
            # Features adicionais
            features = np.array([media_movel, std_movel, max_local, min_local, 
                               amplitude, tendencia, derivada_mean, derivada_std, 
                               segunda_derivada_mean])
            
            # Combinar features originais com as novas features
            X.append(np.concatenate([sequencia, features]))
        else:
            X.append(sequencia)
            
        y.append(series[i + look_back + passo - 1])
    
    return np.array(X), np.array(y)

def treinar_rf_para_componentes(caminhos_componentes, passos=[1], rf_params=None, look_back=None):
    """
    Treina modelos RandomForest para componentes wavelet.
    
    Args:
        caminhos_componentes: Dicionário com nome do componente e caminho do arquivo CSV
        passos: Lista de horizontes de previsão (t+n)
        rf_params: Dicionário com parâmetros para o RandomForestRegressor
        look_back: Valor de look_back para a janela de histórico. Se None, será calculado com base no passo.
    
    Returns:
        Dicionário com metadados dos modelos treinados
    """
    # Tempo de início
    tempo_inicio = time.time()
    
    # Parâmetros padrão do RandomForest se não forem especificados
    if rf_params is None:
        rf_params = {
            'n_estimators': 200,
            'max_depth': 15,
            'min_samples_split': 5,
            'min_samples_leaf': 1,
            'max_features': 'sqrt',
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
    
    # Criar diretórios necessários
    os.makedirs("modelosRF", exist_ok=True)
    os.makedirs("scalersRF", exist_ok=True)
    os.makedirs("configsRF", exist_ok=True)
    
    # Dicionário para armazenar metadados de todos os modelos
    metadados_modelos = {}
    
    # Exibir configuração
    print("\n⚙️ Configuração de Treinamento:")
    print(f"   - Componentes: {list(caminhos_componentes.keys())}")
    print(f"   - Horizontes de previsão: {passos}")
    print(f"   - Parâmetros RF:")
    for param, valor in rf_params.items():
        print(f"      - {param}: {valor}")
    
    # Percorrer cada componente
    for comp, caminho_csv in caminhos_componentes.items():
        tempo_componente = time.time()
        print(f"\n🔍 Processando componente: {comp.upper()}")
        
        try:
            # Carregar dados
            df = pd.read_csv(caminho_csv)
            
            # Garantir que o componente exista no DataFrame
            coluna_componente = comp.upper()
            if coluna_componente not in df.columns:
                print(f"⚠️ Coluna {coluna_componente} não encontrada em {caminho_csv}")
                continue
                
            serie = df[coluna_componente].values.reshape(-1, 1)
            
            # Limpeza e interpolação de dados faltantes
            serie = pd.Series(serie.flatten()).interpolate().ffill().bfill().values.reshape(-1, 1)
            
            # Verificar se há valores infinitos ou NaN
            if np.isnan(serie).any() or np.isinf(serie).any():
                print(f"⚠️ Valores NaN ou infinitos encontrados em {comp}. Aplicando limpeza...")
                serie = np.nan_to_num(serie, nan=np.nanmean(serie), posinf=np.nanmax(serie), neginf=np.nanmin(serie))
            
            # Tamanho do dataset
            print(f"   - Tamanho do dataset: {len(serie)} pontos")
            
            # Metadados específicos do componente
            metadados_modelos[comp] = {}
            
            # Treinar para cada horizonte de previsão
            for passo in passos:
                tempo_passo = time.time()
                
                # Calcular look_back se não foi especificado
                lookback_atual = look_back if look_back is not None else max(10, passo // 2)
                
                print(f"\n🌲 Treinando RandomForest para {comp.upper()} | t+{passo} (look_back={lookback_atual})")
                
                # Usar RobustScaler para lidar melhor com outliers
                scaler = RobustScaler()
                serie_scaled = scaler.fit_transform(serie)
                
                # Salvar scaler
                scaler_filename = f"scalersRF/scaler_{comp}_t{passo}.joblib"
                joblib.dump(scaler, scaler_filename)
                
                # Criar dataset com features adicionais
                X, y = criar_dataset_multi_step(serie_scaled, look_back=lookback_atual, passo=passo, adicionar_features=True)
                
                # Informações sobre o dataset
                print(f"   - Dataset: {X.shape[0]} amostras, {X.shape[1]} features")
                
                # Divisão treino/teste
                split = int(0.8 * len(X))
                X_train = X[:split]
                y_train = y[:split]
                
                # Dicionário para armazenar metadados do modelo
                modelo_info = {
                    'componente': comp,
                    'passo': passo,
                    'look_back': lookback_atual,
                    'num_features': X.shape[1],
                    'num_amostras_treino': len(X_train),
                    'data_treinamento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'usa_features_adicionais': True,
                    'parametros': rf_params
                }
                
                # Criar e treinar modelo com os parâmetros fornecidos
                print(f"   - Iniciando treinamento...")
                model = RandomForestRegressor(**rf_params)
                model.fit(X_train, y_train.ravel())
                
                # Se OOB score estiver disponível, salvar
                if hasattr(model, 'oob_score_'):
                    modelo_info['oob_score'] = float(model.oob_score_)
                    print(f"   - OOB Score: {model.oob_score_:.4f}")
                
                # Calcular e mostrar importância das features
                if len(model.feature_importances_) == X.shape[1]:
                    # Número base de features originais
                    num_features_orig = lookback_atual
                    
                    # Importância das features originais (valores passados)
                    imp_features_orig = model.feature_importances_[:num_features_orig].sum()
                    
                    # Importância das features engineered
                    imp_features_eng = model.feature_importances_[num_features_orig:].sum()
                    
                    print(f"   - Importância das features:")
                    print(f"      - Features originais: {imp_features_orig:.4f} ({imp_features_orig*100:.1f}%)")
                    print(f"      - Features engineered: {imp_features_eng:.4f} ({imp_features_eng*100:.1f}%)")
                    
                    # Top 5 features mais importantes
                    top_indices = np.argsort(model.feature_importances_)[-5:][::-1]
                    
                    # Nomes das features
                    feature_names = [f"t-{i+1}" for i in range(lookback_atual)]
                    feature_names.extend(["média", "std", "max", "min", "amplitude", 
                                          "tendência", "derivada_média", "derivada_std", 
                                          "2ª_derivada_média"])
                    
                    print(f"      - Top 5 features:")
                    for i, idx in enumerate(top_indices):
                        if idx < len(feature_names):
                            print(f"         {i+1}. {feature_names[idx]}: {model.feature_importances_[idx]:.4f}")
                        else:
                            print(f"         {i+1}. Feature {idx}: {model.feature_importances_[idx]:.4f}")
                    
                    # Salvar importância das features no modelo_info
                    modelo_info['importancia_features'] = {
                        'features_originais': float(imp_features_orig),
                        'features_engineered': float(imp_features_eng),
                        'top_5_indices': top_indices.tolist(),
                        'top_5_valores': model.feature_importances_[top_indices].tolist(),
                        'todas_importancias': model.feature_importances_.tolist()
                    }
                
                # Salvar modelo
                modelo_filename = f"modelosRF/rf_{comp}_t{passo}.joblib"
                joblib.dump(model, modelo_filename)
                
                # Salvar metadados do modelo
                config_filename = f"configsRF/config_{comp}_t{passo}.json"
                with open(config_filename, 'w') as f:
                    json.dump(modelo_info, f, indent=4)
                
                # Adicionar informação ao dicionário global
                metadados_modelos[comp][f't{passo}'] = modelo_info
                
                # Tempo de treinamento para este horizonte
                tempo_passo_fim = time.time() - tempo_passo
                print(f"✅ Modelo salvo: {modelo_filename}")
                print(f"📝 Configuração salva: {config_filename}")
                print(f"⏱️ Tempo de treinamento para t+{passo}: {tempo_passo_fim:.2f} segundos")
        
        except Exception as e:
            print(f"❌ Erro ao processar componente {comp}: {str(e)}")
        
        # Tempo para este componente
        tempo_componente_fim = time.time() - tempo_componente
        print(f"⏱️ Tempo total para componente {comp.upper()}: {tempo_componente_fim:.2f} segundos")
    
    # Salvar metadados globais
    with open("configsRF/metadados_global.json", 'w') as f:
        json.dump(metadados_modelos, f, indent=4)
    
    print(f"\n📊 Metadados de todos os modelos salvos em: configsRF/metadados_global.json")
    
    # Tempo total
    tempo_total = time.time() - tempo_inicio
    minutos = int(tempo_total // 60)
    segundos = int(tempo_total % 60)
    print(f"⏱️ Tempo total de execução: {minutos} minutos e {segundos} segundos")
    
    return metadados_modelos

def carregar_config_modelo(componente, passo):
    """Carrega a configuração de um modelo específico"""
    try:
        with open(f"configsRF/config_{componente}_t{passo}.json", 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"❌ Configuração não encontrada para {componente}_t{passo}")
        return None

# --------------------------------
# CÓDIGO PRINCIPAL PARA EXECUÇÃO
# --------------------------------

if __name__ == "__main__":
    print("🚀 Treinamento de RandomForest para componentes Wavelet de detalhe (D1, D2, D3)")
    
    # Definir os caminhos dos arquivos CSV para cada componente de detalhe
    caminhos_componentes = {
        'd1': 'D1_component.csv',
        'd2': 'D2_component.csv',
        'd3': 'D3_component.csv'
    }
    
    # Menu para seleção de configuração
    print("\n📋 Escolha uma configuração:")
    print("1. Configuração básica (rápida)")
    print("2. Configuração para detecção de picos")
    print("3. Configuração personalizada")
    
    opcao = input("Opção (1, 2 ou 3): ")
    
    # Definir parâmetros com base na escolha
    if opcao == "1":
        # Configuração básica e rápida
        rf_params = {
            'n_estimators': 100,
            'max_depth': 10,
            'min_samples_split': 5,
            'min_samples_leaf': 2,
            'max_features': 'sqrt',
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
        look_back = 10
        passos = [1]  # Apenas t+1 para teste rápido
        
    elif opcao == "2":
        # Configuração otimizada para detecção de picos
        rf_params = {
            'n_estimators': 300,
            'max_depth': 15,
            'min_samples_split': 5,
            'min_samples_leaf': 1,  # 1 para maior sensibilidade a outliers (picos)
            'max_features': None,   # Usar todas as features
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
        look_back = 15
        passos = [1, 7]  # t+1 e t+7 como amostra
        
    else:  # Opção 3 ou qualquer outra entrada
        # Configuração personalizada
        print("\n⚙️ Configure os parâmetros do RandomForest:")
        
        # Número de árvores
        try:
            n_estimators = int(input("Número de árvores (recomendado: 100-500) [200]: ") or "200")
        except ValueError:
            n_estimators = 200
            print("Valor inválido. Usando 200 árvores.")
        
        # Profundidade máxima
        try:
            max_depth_input = input("Profundidade máxima (recomendado: 10-20, 'None' para ilimitado) [15]: ") or "15"
            max_depth = None if max_depth_input.lower() == 'none' else int(max_depth_input)
        except ValueError:
            max_depth = 15
            print("Valor inválido. Usando profundidade 15.")
        
        # min_samples_split
        try:
            min_samples_split = int(input("min_samples_split (recomendado: 2-10) [5]: ") or "5")
        except ValueError:
            min_samples_split = 5
            print("Valor inválido. Usando 5.")
        
        # min_samples_leaf
        try:
            min_samples_leaf = int(input("min_samples_leaf (1 para mais sensibilidade a picos) [1]: ") or "1")
        except ValueError:
            min_samples_leaf = 1
            print("Valor inválido. Usando 1.")
        
        # max_features
        max_features_input = input("max_features ('sqrt', 'log2', 'None' para todas) [sqrt]: ") or "sqrt"
        if max_features_input.lower() == 'none':
            max_features = None
        else:
            max_features = max_features_input if max_features_input in ['sqrt', 'log2'] else 'sqrt'
        
        # Look-back
        try:
            look_back_input = input("look_back (recomendado: 10-20, 'None' para automático) [None]: ") or "None"
            look_back = None if look_back_input.lower() == 'none' else int(look_back_input)
        except ValueError:
            look_back = None
            print("Valor inválido. Usando look_back automático.")
        
        # Horizontes de previsão
        passos_input = input("Horizontes de previsão (separados por vírgula, ex: 1,7,30) [1]: ") or "1"
        try:
            passos = [int(p.strip()) for p in passos_input.split(",")]
        except ValueError:
            passos = [1]
            print("Valor inválido. Usando apenas horizonte t+1.")
        
        # Montar parâmetros
        rf_params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'max_features': max_features,
            'bootstrap': True,
            'random_state': 42,
            'n_jobs': -1
        }
    
    # Confirmar configuração
    print("\n⚙️ Configuração selecionada:")
    print(f"   - RandomForest: {rf_params}")
    print(f"   - look_back: {'Automático' if look_back is None else look_back}")
    print(f"   - Horizontes: {passos}")
    
    confirma = input("\nConfirma esta configuração? (s/n): ").lower()
    
    if confirma == 's':
        # Inicia o treinamento
        print("\n🏃 Iniciando treinamento...")
        metadados = treinar_rf_para_componentes(caminhos_componentes, passos, rf_params, look_back)
        
        # Exibir resumo
        print("\n✅ Treinamento concluído!")
        print("📊 Resumo dos modelos treinados:")
        
        for comp, modelos in metadados.items():
            print(f"\n📌 Componente: {comp.upper()}")
            for passo_key, info in modelos.items():
                print(f"   - Passo {info['passo']}: look_back={info['look_back']}, features={info['num_features']}")
                if 'oob_score' in info:
                    print(f"     OOB Score: {info['oob_score']:.4f}")
    else:
        print("Treinamento cancelado.")